# C++ 文件实现算子

In [ ]:
#include <openvino/op/op.hpp>
class MyCustomOp : public ov::op::Op {
public:
    OPENVINO_OP("MyCustomOp", "custom_opset");
    MyCustomOp() = default;
    MyCustomOp(const ov::Output<ov::Node>& x);
    void validate_and_infer_types() override;
    std::shared_ptr<ov::Node> clone_with_new_inputs(const ov::OutputVector& new_args) const override;
    bool visit_attributes(ov::AttributeVisitor& visitor) override;
    bool evaluate(ov::TensorVector& outputs, const ov::TensorVector& inputs) const override;
    bool has_evaluate() const override;
private:
    size_t m_input_shape_size = 0;
};

MyCustomOp::MyCustomOp(const ov::Output<ov::Node>& x) {
    constructor_validate_and_infer_types();
    set_input(0, x);
}

void MyCustomOp::validate_and_infer_types() {
    auto input_shape = get_input_partial_shape(0);
    auto input_rank = input_shape.rank().get_length();
    set_output_type(0, get_input_element_type(0), get_input_partial_shape(0));
}

std::shared_ptr<ov::Node> MyCustomOp::clone_with_new_inputs(const ov::OutputVector& new_args) const {
    return std::make_shared<MyCustomOp>(new_args.at(0));
}

bool MyCustomOp::visit_attributes(ov::AttributeVisitor& visitor) {
    return true;
}

bool MyCustomOp::evaluate(ov::TensorVector& outputs, const ov::TensorVector& inputs) const {
    auto x = inputs[0];
    auto output_shape = x.get_partial_shape();
    auto output_rank = output_shape.rank().get_length();
    auto element_type = x.get_element_type();
    auto& backend = *ov::runtime::Backend::get();

    // 只支持 float 和 double 类型的输入
    if (element_type != ov::element::f32 && element_type != ov::element::f64) {
        throw ov::Exception("MyCustomOp only accepts float and double inputs.");
    }

    if (element_type == ov::element::f32) {
        auto output_tensor = ov::runtime::Tensor(ov::element::f32, output_shape);
        auto sin_x = backend.sin(ov::runtime::TensorView<float>{x});
        auto output_tensor_view = ov::runtime::TensorView<float>{output_tensor};
        auto output_ptr = output_tensor_view.data();
        auto sin_x_ptr = sin_x.data();
        auto size = output_shape[0].get_length();
        for (size_t i = 0; i < size; ++i) {
            *output_ptr++ = 2 * sin_x_ptr[i] + 1;
        }
        outputs.push_back(output_tensor);
    } else {
        auto output_tensor = ov::runtime::Tensor(ov::element::f64, output_shape);
        auto sin_x = backend.sin(ov::runtime::TensorView<double>{x});
        auto output_tensor_view = ov::runtime::TensorView<double>{output


# 复杂自定义：以max_pool为例

In [6]:
#pragma once
#include <limits>
#include "openvino/op/util/max_pool_base.hpp"

namespace ov {
namespace op {
namespace v8 {

class OPENVINO_API MaxPool : public op::util::MaxPoolBase {
public:
    //宏
    OPENVINO_OP("MaxPool", "opset8", op::util::MaxPoolBase);
    BWDCMP_RTTI_DECLARATION;
    MaxPool() = default;

    /// 构造函数
    ///
    /// \param arg 
    /// \param strides 
    /// \param dilations 
    /// \param pads_begin 
    /// \param pads_end 
    /// \param kernel 
    /// \param rounding_type 
    /// \param auto_pad 
    /// \param index_element_type 
    /// \param axis 
    ///Strides 和Shape都是列表
    MaxPool(const Output<Node>& arg,
            const Strides& strides,
            const Strides& dilations,
            const Shape& pads_begin,
            const Shape& pads_end,
            const Shape& kernel,
            const op::RoundingType rounding_type = op::RoundingType::FLOOR,
            const PadType auto_pad = op::PadType::EXPLICIT,
            const element::Type index_element_type = element::i64,
            const int64_t axis = 0);

    bool visit_attributes(AttributeVisitor& visitor) override;
    void validate_and_infer_types() override;

    std::shared_ptr<Node> clone_with_new_inputs(const OutputVector& new_args) const override;

    // 属性获取函数
    const Strides& get_dilations() const noexcept {
        return m_dilations;
    }
    //属性修改函数
    void set_dilations(const Strides& dilations) {
        m_dilations = dilations;
    }

    // 属性获取函数
    element::Type get_index_element_type() const noexcept {
        return m_index_element_type;
    }
    //属性修改函数
    void set_index_element_type(const element::Type index_element_type) {
        m_index_element_type = index_element_type;
    }

    // 属性获取函数
    int64_t get_axis() const {
        return m_axis;
    }
    //属性修改函数
    void set_axis(const int64_t axis) {
        m_axis = axis;
    }

    bool has_evaluate() const override;
    OPENVINO_SUPPRESS_DEPRECATED_START
    bool evaluate(const HostTensorVector&, const HostTensorVector&) const override;
    OPENVINO_SUPPRESS_DEPRECATED_END

private:
    Strides m_dilations;
    element::Type m_index_element_type{element::i64};
    int64_t m_axis{0};
};
}  // namespace v8
}  // namespace op
}  // namespace ov


SyntaxError: invalid syntax (Temp/ipykernel_13288/1107137848.py, line 1)

In [ ]:
#include "ngraph/op/max_pool.hpp"
#include "itt.hpp"
#include "ngraph/attribute_visitor.hpp"
#include "ngraph/op/add.hpp"
#include "ngraph/op/constant.hpp"
#include "ngraph/runtime/host_tensor.hpp"
#include "ngraph/runtime/reference/max_pool.hpp"
#include "ngraph/validation_util.hpp"

using namespace std;
using namespace ngraph;

bool op::v1::MaxPool::update_auto_padding(const PartialShape& in_shape,
                                          Shape& new_pads_end,
                                          Shape& new_pads_begin) const
{
    bool update_auto_padding_succeed = true;
    if (m_auto_pad == PadType::SAME_UPPER || m_auto_pad == PadType::SAME_LOWER)
    {
        CoordinateDiff pads_end, pads_begin;
        update_auto_padding_succeed =
            try_apply_auto_padding(in_shape,
                                   m_kernel,
                                   m_strides,
                                   Strides(m_kernel.size(), 1), // No dilation
                                   m_auto_pad,
                                   pads_end,
                                   pads_begin);
        new_pads_end = Shape(pads_end.begin(), pads_end.end());
        new_pads_begin = Shape(pads_begin.begin(), pads_begin.end());
    }
    return update_auto_padding_succeed;
}

NGRAPH_RTTI_DEFINITION(op::v1::MaxPool, "MaxPool", 1);
//带参构造
op::v1::MaxPool::MaxPool(const Output<Node>& arg,
                         const Strides& strides,
                         const Shape& pads_begin,
                         const Shape& pads_end,
                         const Shape& kernel,
                         op::RoundingType rounding_type,
                         const PadType& auto_pad)
    : Op({arg})
    , m_kernel(kernel)
    , m_strides(strides)
    , m_pads_begin(pads_begin)
    , m_pads_end(pads_end)
    , m_auto_pad(auto_pad)
    , m_rounding_type(rounding_type)
{
    //固定，调用的是validate_and_infer_types
    constructor_validate_and_infer_types();
}

In [ ]:
//访问所有的属性，对所有的属性都要使用
bool ngraph::op::v1::MaxPool::visit_attributes(AttributeVisitor& visitor)
{   //作用域名，格式就是他的函数名
    NGRAPH_OP_SCOPE(v1_MaxPool_visit_attributes);
    visitor.on_attribute("strides", m_strides);
    visitor.on_attribute("pads_begin", m_pads_begin);
    visitor.on_attribute("pads_end", m_pads_end);
    visitor.on_attribute("kernel", m_kernel);
    visitor.on_attribute("rounding_type", m_rounding_type);
    visitor.on_attribute("auto_pad", m_auto_pad);
    return true;
}

In [ ]:
void op::v1::MaxPool::validate_and_infer_types()
{
    NGRAPH_OP_SCOPE(v1_MaxPool_validate_and_infer_types);
    if (0 == m_strides.size())
    {
        m_strides = Strides(m_kernel.size(), 1);
    }

    if (0 == m_pads_begin.size())
    {
        m_pads_begin = Shape(m_kernel.size(), 0);
    }

    if (0 == m_pads_end.size())
    {
        m_pads_end = Shape(m_kernel.size(), 0);
    }

    const PartialShape& arg_shape = get_input_partial_shape(0);

    NODE_VALIDATION_CHECK(this,
                          arg_shape.rank().compatible(3) || arg_shape.rank().compatible(4) ||
                              arg_shape.rank().compatible(5),
                          "Expected a 3D, 4D or 5D tensor for the input. Got: ",
                          arg_shape);

    if (arg_shape.rank().is_static())
    {
        NODE_VALIDATION_CHECK(this,
                              static_cast<int64_t>(m_pads_end.size()) ==
                                  arg_shape.rank().get_max_length() - 2,
                              "Expected pads_end size to be equal to input size - 2. Got: ",
                              m_pads_end.size());

        NODE_VALIDATION_CHECK(this,
                              static_cast<int64_t>(m_pads_begin.size()) ==
                                  arg_shape.rank().get_max_length() - 2,
                              "Expected pads_begin size to be equal to input size - 2. Got: ",
                              m_pads_begin.size());
        NODE_VALIDATION_CHECK(this,
                              static_cast<int64_t>(m_kernel.size()) ==
                                  arg_shape.rank().get_max_length() - 2,
                              "Expected kernel size to be equal to input size - 2. Got: ",
                              m_kernel.size());
        NODE_VALIDATION_CHECK(this,
                              static_cast<int64_t>(m_pads_end.size()) ==
                                  arg_shape.rank().get_max_length() - 2,
                              "Expected strides size to be equal to input size - 2. Got: ",
                              m_strides.size());
    }

    auto output_shape = PartialShape::dynamic();
    if (arg_shape.rank().is_static())
    {
        output_shape =
            std::vector<Dimension>(arg_shape.rank().get_max_length(), Dimension::dynamic());
        if (arg_shape[0].is_static())
        {
            output_shape[0] = arg_shape[0]; // batch size
        }
        if (arg_shape[1].is_static())
        {
            output_shape[1] = arg_shape[1]; // channel size
        }
    }

    bool update_auto_padding_succeed = true;
    if (m_auto_pad == PadType::SAME_UPPER || m_auto_pad == PadType::SAME_LOWER)
    {
        update_auto_padding_succeed = update_auto_padding(arg_shape, m_pads_end, m_pads_begin);
    }
    if (m_auto_pad == PadType::VALID)
    {
        m_pads_end = Shape(m_pads_end.size(), 0);
        m_pads_begin = Shape(m_pads_begin.size(), 0);
    }
    // infer_batched_forward_pooling wants CoordinateDiffs for these, while the pooling ops for
    // now still take Shape (no negative padding).
    CoordinateDiff pads_begin(m_pads_begin.begin(), m_pads_begin.end());
    CoordinateDiff pads_end(m_pads_end.begin(), m_pads_end.end());

    set_output_type(0,
                    get_input_element_type(0),
                    update_auto_padding_succeed
                        ? infer_batched_pooling_forward(this,
                                                        arg_shape,
                                                        pads_begin,
                                                        pads_end,
                                                        m_kernel,
                                                        m_strides,
                                                        true,
                                                        m_rounding_type == op::RoundingType::CEIL)
                        : output_shape);
}


shared_ptr<Node> op::v1::MaxPool::get_default_value() const
{
    return op::Constant::create(get_element_type(), get_shape(), {0});
}

In [ ]:
//创建一个新的OP，把参数传递给他即可
shared_ptr<Node> op::v1::MaxPool::clone_with_new_inputs(const OutputVector& new_args) const
{  //作用域
    NGRAPH_OP_SCOPE(v1_MaxPool_clone_with_new_inputs);
    check_new_args_count(this, new_args);
    //把参数传递进去
    return make_shared<v1::MaxPool>(
        new_args.at(0), m_strides, m_pads_begin, m_pads_end, m_kernel, m_rounding_type, m_auto_pad);
}

In [7]:
//自定义的OP，都是以这种形式来实现
namespace maxpool
{
    template <element::Type_t ET>
    inline bool evaluate(const HostTensorPtr& arg,
                         const HostTensorPtr& out,
                         const Shape& out_shape,
                         const Shape& window_shape,
                         const Strides& window_movement_strides,
                         const Shape& padding_below,
                         const Shape& padding_above)
    {
        using T = typename element_type_traits<ET>::value_type;
        out->set_shape(out_shape);
        //这个是已经有的实现函数
        runtime::reference::max_pool<T>(arg->get_data_ptr<ET>(),
                                        out->get_data_ptr<ET>(),
                                        arg->get_shape(),
                                        out_shape,
                                        window_shape,
                                        window_movement_strides,
                                        padding_below,
                                        padding_above);
        return true;
    }

    bool evaluate_maxpool(const HostTensorPtr& arg,
                          const HostTensorPtr& out,
                          const Shape& out_shape,
                          const Shape& kernel,
                          const Strides& strides,
                          const Shape& pad_begin,
                          const Shape& pad_end)
    {
        bool rc = true;
        auto arg_shape = arg->get_shape();
        //根据输出的类型来选择调用的函数
        switch (out->get_element_type())
        {// NGRAPH_TYPE_CASE就是调用evaluate函数的意思，
            NGRAPH_TYPE_CASE(evaluate_maxpool,i32,arg,out,out_shape,kernel,strides,pad_begin,pad_end);
            NGRAPH_TYPE_CASE(evaluate_maxpool,i64,arg,out,out_shape,kernel,strides,pad_begin,pad_end);
            NGRAPH_TYPE_CASE(evaluate_maxpool, u32, arg, out, out_shape, kernel, strides, pad_begin, pad_end);
            NGRAPH_TYPE_CASE(evaluate_maxpool, u64, arg, out, out_shape, kernel, strides, pad_begin, pad_end);
            NGRAPH_TYPE_CASE(evaluate_maxpool, f16, arg, out, out_shape, kernel, strides, pad_begin, pad_end);
            NGRAPH_TYPE_CASE(evaluate_maxpool, f32, arg, out, out_shape, kernel, strides, pad_begin, pad_end);
        default: rc = false; break;
        }
        return rc;
    }
} // namespace maxpool

bool op::v1::MaxPool::evaluate_maxpool(const HostTensorVector& outputs,
                                       const HostTensorVector& inputs) const
{
    auto arg_shape = inputs[0]->get_partial_shape();
    
    //必须实现获得属性的函数
    auto pads_begin_s = get_pads_begin();
    auto pads_end_s = get_pads_end();
    
    update_auto_padding(arg_shape, pads_begin_s, pads_end_s);
    CoordinateDiff pads_begin(pads_begin_s.begin(), pads_begin_s.end());
    CoordinateDiff pads_end(pads_end_s.begin(), pads_end_s.end());
    auto out_shape = infer_batched_pooling_forward(this,
                                                   arg_shape,
                                                   pads_begin,
                                                   pads_end,
                                                   get_kernel(),
                                                   get_strides(),
                                                   true,
                                                   get_rounding_type() == op::RoundingType::CEIL);
    //获得属性的函数就是在这里调用的
    return maxpool::evaluate_maxpool(inputs[0],
                                     outputs[0],
                                     out_shape.get_shape(),
                                     get_kernel(),
                                     get_strides(),
                                     get_pads_begin(),
                                     get_pads_end());
}

SyntaxError: invalid syntax (Temp/ipykernel_13288/4027719477.py, line 10)

In [ ]:
//输入的都是常列表引用
bool op::v1::MaxPool::evaluate(const HostTensorVector& outputs,
                               const HostTensorVector& inputs) const
{
    //版本_FUNC_has_evaluate  = v0_Log_has_evaluate，保证和has_evaluate里的一样
    NGRAPH_OP_SCOPE(v1_MaxPool_evaluate);
    //要写自定义的OP执行函数，在这里调用
    return evaluate_maxpool(outputs, inputs);
}

In [8]:
//当输入是指定数据类型时可以执行，是固定的形式,这是和evaluate配套出现的函数
//只要修改作用域标识符即可 ： 版本_FUNC_has_evaluate  = v0_Log_has_evaluate
//按照输入的类型确定是否可以执行evaluate
bool op::v1::MaxPool::has_evaluate() const
{
    //标识符
    NGRAPH_OP_SCOPE(v0_Log_has_evaluate)
    
    switch (get_input_element_type(0))
    {
    //哪些类型可以执行evaluate
    case ngraph::element::i32:
    case ngraph::element::i64:
    case ngraph::element::u32:
    case ngraph::element::u64:
    case ngraph::element::f16:
    case ngraph::element::f32: return true;
    default: break;
    }
    return false;
}

SyntaxError: invalid syntax (Temp/ipykernel_13288/3756834844.py, line 1)

# python 生成MO 的识别Extensions
    OP只是一个属性保存的，使用OP给Node提供属性而已
共三步
1. 将OP加入到Model Optimizer中以便于生成IR文件
2. 在NGraph中执行op，可以云寻
3. 在IE中让不同的设备都可以执行
第一步

    分成三类
1.Model Optimizer operation.基类构建
2.A framework operation extractor.特征提取器
3.front, middle or back 阶段的变形

In [ ]:
命令 mo -->>subprocess_main-->>main_onnx.py-->>main-->>_convert-->>driver----
                    get_onnx_cli_parser解析参数--|


------>>prepare_ir----->>unified_pipeline 
                            建立一个图，对图进行四个操作[load,front,mid,back]
                            get_replacers_order：先对四个操作进行排序
                                _registered_classes_dict全局字典中保存所有类，对他们进行图排序
                    
    
                            apply_replacements_list：遍历图的所有节点，每个类都要
                                    进行对应的find_and_replace_pattern
            
                                    force_clean_up---->>clean_up
                
                                    shape_inference(graph)-->>infer

    
----->>emit_ir

### mo_extension/ops/cosh.py

_registered_classes_dict全局字典中保存了所有的ops,front,middle,back类
有Op, Activation, Elementwise, UnaryElementwise, LogicalElementwise,EmbeddingBagBase, ReduceOp, Scatter, ScatterNDBase, FFTBase这些父类，需要取找所有子类放入到字典中，导入所有文件


In [1]:
from mo.front.common.partial_infer.elemental import copy_shape_infer
from mo.graph.graph import Graph, Node
from mo.ops.op import Op
from mo.ops.pooling import Pooling
import numpy as np
from extensions.ops.argmax import arg_ops_infer


class ArgMinOp(Op):
    version = 'opset1'
    op = 'ArgMin'
    enabled = False                          #表示该OP是否会被MO使用或者包括
    operation = None                         #简单的实现OP方法,写了这个可以不再用C++编写具体实现
    #operation = staticmethod(lambda x: 1 / (1 + np.exp(-x)))   可以直接写出计算OP
    def __init__(self, graph: Graph, attrs: dict):
        mandatory_props = {
            'type':self.op,
            'op': self.op,
            'infer': self.infer,   
            'operation': self.operation,      
            'type_infer': self.type_infer,            #节点的结构属性
            'output_type': np.int64,
            'in_ports_count': 2,
            'out_ports_count': 1,
        }#这些都是默认的,给的是两个attrs字典
        super().__init__(graph, mandatory_props, attrs)
    
    def supported_attrs(self):
        return [
            'top_k',
            'axis',
        ]#这个方法返回列表，其存储的是需要保存到IR中attrs，从上面的初始化中来
    
    @staticmethod
    def infer(node):                      #  计算输出的形状
        """
        静态方法或者类方法，传入的是自身OP构造的Node
        该方法是用来计算输出的形状，类型等张量信息，需要用到node自身的port中所保存data
        node.inport(2).data.get_shape
        或者node.in_node(2).shape
        而data本身包含的六个方法对应处理其属性,
        get_shape()
        set_shape()
        get_value()
        set_value()
        get_attr()
        set_attr()
        value,shape,type
    
        最终的结果就是用这六个方法来计算的得到
        
        通过设置输出port中data的形状完成set_shape，形状是np数组set_value
        
        在这里也可以计算
        shape = x
        value =c
        node.out_port(1).data.set_shape(shape)
        node.out_port(1).data.set_value(value)
        
        或者用这种更简单的方法
        node.out_node(0).shape = shape
        node.out_node(0).value = value
        """
    @staticmethod
    def type_infer(node):
        """
        类型推断set_data_type
        """
    def backend_attrs(self): #这个是用于保存到IR中的属性，并不是全有
        return [
            ('strides', lambda node: ','.join(map(str, node['stride'][node.spatial_dims]))),
            ('kernel', lambda node: ','.join(map(str, node['window'][node.spatial_dims]))),
            ('pads_begin', lambda node: ','.join(map(str, get_backend_pad(node.pad, node.spatial_dims, 0))))]

ModuleNotFoundError: No module named 'mo'

# 2 Operation Extractor
### mo_extension/front/onnx/cosh_ext.py

In [1]:
#这步操作是先于OP的，载入框架模型后就要进行属性提取，用于后续构造图节点
#基类是mo.front.extractor.FrontExtractorOp，用内置类方法extract来获得唯一参数NOde的属性
from extensions.ops.argmin import ArgMinOp    #要把OP类导入进来，进行属性添加
from mo.front.extractor import FrontExtractorOp
from mo.front.onnx.extractors.utils import onnx_attr

#FrontExtractorOp作为基类，
#里面只是定义了
#    registered_ops = {}
#    registered_cls = []
#子类必须实现类方法extract
#
class ArgMinFrontExtractor(FrontExtractorOp):
    op = 'ArgMin'                    #OP名字
    enabled = True                  #MO是否会用到

    @classmethod
    def extract(cls, node):
        #用onnx中的helper函数的onnx_attr来解析onnx模型，读取出OP中的attr，
        #唯一的区别在于不同的框架用不同的解析属性的方法
        keepdims = onnx_attr(node, 'keepdims', 'i', default=1)
        axis = onnx_attr(node, 'axis', 'i', default=0)
        #制作成字典
        attrs = {
            'axis': axis,
            'top_k': 1,
            'keepdims': keepdims,
            'remove_values_output': True
        }
        #将其当作图节点，传入图中对应的节点
        ArgMinOp.update_node_stat(node, attrs)
        return cls.enabled


ModuleNotFoundError: No module named 'extensions'

In [ ]:
#将自定义算子MyRelu ，使用OPENVINO自带的RELU进行替换表示  1 VS 1
core.add_extension(ov::frontend::OpExtension<>("Relu", "MyRelu"));
#或者
core.add_extension(ov::frontend::OpExtension<>(ov::opset8::Relu , "MyRelu"));

                                        
core.add_extension(ov::frontend::OpExtension<CustomOperation>(
    { 
        #实现属性的复制使得两个属性数据一样a==b，{“a”,“b”}
    {"attr1", "fw_attr1"}, 
     {"attr2", "fw_attr2"} },
    {}
));

In [ ]:
#include <openvino/opsets/opset8.hpp>
core.add_extension(ov::frontend::ConversionExtension(
    "ThresholdedReLU",#ONNX中的算子名称
    [](const ov::frontend::NodeContext& node) {
        auto greater = std::make_shared<ov::opset8::Greater>(
            node.get_input(0),
            ov::opset8::Constant::create(ov::element::f32, {}, {node.get_attribute<float>("alpha")}));
        auto casted = std::make_shared<ov::opset8::Convert>(greater, ov::element::f32);
        return ov::OutputVector{ std::make_shared<ov::opset8::Multiply>(node.get_input(0), casted) };
    }));
